In [ ]:
# 필요한 라이브러리 import
import re # 정규표현식을 사용하기 위한 모듈
import torch # PyTorch를 사용하여 딥러닝 모델 처리
from transformers import MllamaForConditionalGeneration, MllamaProcessor # Mllama 모델과 Processor를 가져옴

In [ ]:
# 사전 학습된 Mllama 모델 로드 (!!!GPU 권장!!!)
model = MllamaForConditionalGeneration.from_pretrained(
    'Bllossom/llama-3.2-Korean-Bllossom-AICA-5B', # 사전 학습된 모델의 이름
    torch_dtype=torch.bfloat16,                   # bfloat16 데이터 타입을 사용하여 메모리 효율 향상
    device_map='auto'                             # 모델을 자동으로 적절한 디바이스(GPU/CPU)에 매핑
)

# Mllama Processor 초기화
processor = MllamaProcessor.from_pretrained('Bllossom/llama-3.2-Korean-Bllossom-AICA-5B')  # 동일한 모델 이름의 프로세서 로드

# 텍스트 생성 함수 정의
def generate(mode, text):
    """
    입력 텍스트와 생성 모드에 따라 요약 또는 퀴즈를 생성하는 함수.
    
    Parameters:
    mode (str): 'summary' 또는 'quiz' 중 하나로 텍스트 생성 방식을 지정
    text (str): 처리할 입력 텍스트

    Returns:
    str: 모델이 생성한 결과 텍스트
    """
    # 생성 모드에 따라 프롬프트 설정
    if mode == "summary":
        prompt = "다음 텍스트를 요약해줘: "
    elif mode == "quiz":
        prompt = "다음 텍스트로 문제와 정답을 1개만 만들어줘: "
    
    # ChatGPT 스타일 메시지 포맷 생성
    messages = [{'role': 'user',                      # 사용자의 역할로 메시지 생성
                 'content': [{'type': 'text',         # 텍스트 타입 명시
                              'text': prompt+text}]}] # 프롬프트와 입력 텍스트를 결합하여 메시지 생성

    # 메시지를 모델이 이해할 수 있는 템플릿으로 변환
    input_text = processor.apply_chat_template(messages,
                                               tokenize=False,             # 토큰화는 나중 단계에서 진행
                                               add_generation_prompt=True) # 추가 프롬프트 활성화

    # 입력 데이터를 모델에 전달 가능한 형태로 변환
    inputs = processor(text=input_text,
                       add_special_tokens=False,             # 특수 토큰 추가 비활성화
                       return_tensors="pt").to(model.device) # 모델이 위치한 디바이스로 이동

    # 모델을 사용해 텍스트 생성
    output = model.generate(**inputs,
                            max_new_tokens=256, # 생성할 최대 토큰 수
                            temperature=0.1,    # 출력 다양성을 낮추기 위한 설정
                            eos_token_id=processor.tokenizer.convert_tokens_to_ids('<|eot_id|>'),  # 종료 토큰 ID 설정
                            use_cache=False)    # 캐싱 비활성화

    # 생성된 토큰을 텍스트로 디코딩
    output_text = processor.decode(output[0])

    # 정규표현식을 사용해 원하는 텍스트 부분 추출
    match = re.search(r'<\|start_header_id\|>assistant<\|end_header_id\|>(.*?)<\|eot_id\|>',
                      output_text, # 모델의 출력 텍스트에서
                      re.DOTALL)   # 여러 줄에 걸쳐 매칭 가능하도록 설정
    
    # 매칭된 그룹 반환
    return match.group(1)

In [ ]:
# 예제 입력 텍스트
text = """
엔비디아가 차세대 게이밍 GPU(그래픽처리장치) '지포스 RTX 50시리즈'를 내놓으며 PC용 AI칩 시장을 겨냥한다. 올해 초 미국 라스베이거스서 개최된 CES 2025에서 선보인 바 있는 RTX 50 시리즈는 개인용 AI PC 시장을 바꿔놓을 게임 체인저로 지목되면서 향후 인텔, 퀄컴, AMD 등과의 치열한 각축전이 예상되고 있다. 20일 엔비디아는 서울 코엑스에서 'RTX AI PC 데이'를 열고 새로운 아키텍처 '블랙웰'이 탑재된 '지포스 RTX 50 시리즈'를 발표했다. 이는 RTX는 엔비디아가 만든 GPU(그래픽처리장치)인데 이번에 새롭게 나온 50 시리즈(5090, 5080, 5070TI, 5070)등은 이전 세대보다 강화된 AI(인공지능) 기술과 낮아진 가격을 자랑한다. 이중에서도 5090 제품의 경우 최고급 PC에 주로 사용되며 게이밍, 그래픽 연산 등에 주로 사용된다. 게이머 뿐 아니라 개발자, 크리에이터 등을 위한 최고 사양을 자랑한다. RTX 5090의 경우 초당 데이터 전송량이 초당 1.8TB(테라바이트)로, 이전 모델인 RTX 4090보다 두 배 향상됐다. 성능은 높아졌으나 배터리 효율은 더욱 개선됐다는 것이 엔비디아 측의 설명이다.
"""

In [ ]:
# 요약 생성 실행
summary = generate("summary", text)
print(summary) # 생성된 요약 출력

In [ ]:
# 요약을 바탕으로 퀴즈 생성 실행
quiz = generate("quiz", summary)
print(quiz) # 생성된 퀴즈 출력